# A/B testing
Today we are going to talk about A/B testing! It's a data driven(experimental) approach to help us determine whether a change(eg. introduction of a new feature, new landing page, etc) brings improvement. In marketing, they often call it "split test".

The definition of improvement depends on the business problems we are trying to solve and the business objective and metrics that we want to optimize for.

Some metrics: 
Conversion rates, Improved engagement, Daily Active Users, App installs, Increased Utilization Rate, Churn Rate, Average order value and more..

# Case study

To make it a little fun, let's do a case study as an example. Let's say we have a new landing page for the product that the company is selling and we want to find out if the new landing page leads to a higher conversion rate.

<img src='ab_testing/ab_testing_img.jpg' width=400/>

So we have an existing landing page(version A) and a proposed landing page(version B) that we want to test. The general idea of this A/B test is to show these two versions to two similarly sized audiences and analyze which one performed better over a specific period of time, using the collected results.


# Business Objectives
The first thing we have to think about is the objective and metrics that matter to the business, as well defined as possible. In this case, its:

1) Conversion rate (Proportion of sessions resulting in a purchase). Obviously, this plays a direct impact in the bottom line of the compant.

2) Average time spent on the landing page per session (Average number of seconds). It determine whether landing page is relevant to the target audience.

# Design of experiment
This could be seen as a scientific experiment where we want to measure the effects of an intervention has, on the control and treatment group, which is visitors to landing page A and B, respectively. Since it's an controlled experiment, we have to clearly define the boundaries of this experiment and evaluate the results using hypothesis test.

## Formulate a hypothesis
Before conducting an A/B testing, we should state the null hypothesis and alternative hypothesis.

Null Hypothesis($H_{0}$): There is no difference between the control and treatment group. In our context, it means there's NO observed difference in the conversion rate between visitors to page A and B.

Alternative hypothesis($H_{1}$): There is a difference between the control and treatment group. In our context, it could be an observed difference in the conversion rate between visitors to page A and B.

## Sampling for control and treatment group
Next, we have to consider how we should draw the samples for control and treatment group. Technically, an A/B test maps users to "treatment"(B) buckets where the experiment is triggered. And the "control" bucket (A) is the current production experience.

<img src='https://res.cloudinary.com/springboard-images/image/upload/q_auto,f_auto,fl_lossy/wordpress/2018/06/Blog-A_Btesting.png' width=500/>


Some considerations:

1) Size of both groups should be the same to ensure a fair comparsion.

2) How should we sample the visitors? Should we draw a random sample or straified sample?
<b>Random sampling</b> ensures that each sample in a population has an equal chance of being chosen and being random, which means without regard to any attribute of the population, it could potentially eliminates sampling bias. 

But what happens if there's some subtle, uneven population attribute distribution. Consider these example:

2.1) If most of the users are using iOS devices, which are likely to use Safari as web browser, would that impact the measurement VS users who are using Android, which are likely to use a diverse set of browsers? Perhaps some of the JS doesn't work for some browsers?

2.2) If the experiment required loading additional JS and making other calls, which made the bucketing for treatment less likely to succeed for users on slow connections. This resulted in bias: users on slow connections were more likely to show up in control than in treatment, subtly skewing the results.

2,3) If we want to measure the impact for Japan visitors, what should the control and treatment group be? In this case, we cannot just randomly sample any visitors but only collect visitors from Japan.

One alternative is <b>Stratified Random Sampling</b>: Stratify the samples into known groups and sample from the  groups independently.

3) How do we ensure that there's no bias in the sampling method?
It’s important to eliminate bias because you want the results of your A/B test to be representative of the entire population rather than the sample itself.

4) What's the sample size?
It’s crucial that we determine the minimum sample size prior to conducting it so that we can eliminate under coverage bias(bias from sampling too few observations). We can calculate it based on:

<b>Effect size</b>: the magnitude of difference between averages of test and control group. It is the variance in averages between test and control groups divided by the standard deviation of the control.

<b>Statistical Power</b>: the probability of finding a statistical significance in our test. 0.8 is a common value.


While the idea of A/B testing is simple to understand, executing all the parts correct is not as straightforward because results could be easily affected if we are not careful how we draw the samples. 

# Evaluating results from A/B test
Lets say we have ran the test and collected such results:
<img src='ab_testing/eg_data.png' width=500/>

From this data, can we conclude that landing page B is better then A since it has a higher conversion rate? 

If yes, how confidence are we in that claim? 

Did a higher CR/higher average time spent of version B happen by chance?

Note that the results we observed are actually derived from samples of a much bigger populations(all possible users). Their statistical properties vary around the ones of the populations they come from.

Going back to the hypothesis that we have formulated:

Null Hypothesis($H_{0}$): There is no difference between the control and treatment group. In our context, it means there's NO observed difference in the conversion rate between visitors to page A and B.

Alternative hypothesis($H_{1}$): There is a difference between the control and treatment group. In our context, it could be an observed difference in the conversion rate between visitors to page A and B.

What we could do next is to:
1. Model $H_{0}$ as a distribution on a single real-valued random variable (called the test statistic)
2. Find out the <b>p-value</b>: How likely the samples, or more extreme ones, could have been generated under $H_{0}$. If it's lower than alpha or significance level, we reject $H_{0}$ and say that there is indeed a signifance difference.


In [1]:
import numpy as np
from scipy import stats
import pandas as pd
from scipy.stats import ttest_ind
from statsmodels.stats.weightstats import ztest
import statsmodels
import matplotlib.pyplot as plt
import random
random.seed(888)
np.random.seed(888)

In [2]:
#we generate a random sample under the binomial distribution which consists of 100 bernouili trials to simulate purchase(1/0) for each visit
n_a, n_b = 100,100
np.random.seed(888)
pur_a = np.random.binomial(n=1, p=0.5,size=n_a)
pur_b = np.random.binomial(n=1, p=0.55,size=n_b)
#generate random normally distribted samples for simulating time spent
time_a = np.random.normal(3, 1, n_a)
time_b = np.random.normal(4, 1, n_b)
grp_type_a = [0] * len(pur_a)
grp_type_b = [1] * len(pur_b)
dataset = pd.DataFrame({'treatment_grp':grp_type_a+grp_type_b,
                        'purchase':list(pur_a)+list(pur_b),
                       'time_spent':list(time_a)+list(time_b)}
                      )
dataset.head()

,treatment_grp,purchase,time_spent
0,0,1,5.407855
1,0,0,4.301718
2,0,0,1.674725
3,0,1,4.687605
4,0,0,3.411356


In [3]:
ab_summary = dataset.pivot_table(values='purchase', index='treatment_grp', aggfunc=np.sum)
# add additional columns to the pivot table
ab_summary['total_visits'] = dataset.pivot_table(values='purchase', index='treatment_grp', aggfunc=lambda x: len(x))
ab_summary['avg_conv_rate'] = dataset.pivot_table(values='purchase', index='treatment_grp') #default=mean
ab_summary['avg_time_spent'] = dataset.pivot_table(values='time_spent', index='treatment_grp')
ab_summary

,purchase,total_visits,avg_conv_rate,avg_time_spent
treatment_grp,,,,
0,47,100,0.47,3.002490
1,52,100,0.52,3.986131


In [4]:
# stats.binom(100, 0.5).pmf(1)

In [5]:
# x = np.arange(stats.binom.ppf(0.01, n, p),
#              stats.binom.ppf(0.99, n, p))

In [6]:
# fig, ax = plt.subplots(figsize=(12,6))
# x = np.linspace(40, 140, 1000)
# y = stats.binom(n_a, 0.5).pmf(x)
# ax.bar(x, y, alpha=0.5)
# #ax.axvline(np.mean(pur_b), c='blue', alpha=0.75, linestyle='--')
# plt.xlabel('converted')
# plt.ylabel('probability')

## T-test for evaluating average time spent
We could run a statistical test but choosing the right one given the underlying data type and assumptions can be a little confusing. If we refer to the flowchart here:
<img src='http://www.datageeko.com/imgs/stats_test_map_v2.png?_t=1615425796'/>
We could see that we have a case of "2-samples" since we have a control and treatment group, and we are testing the the differences in means, we are also dealing with independent samples since the samples don't come from the same group of people. It's arguable and many practitioners would just assume that they know the population std dev and cheated a little by using the sample std dev to estimate the population's. Instead of doing that, we should use the t-test, which models this uncertainty.

To put it simply: For A/B testing, a clear and concise statement is: in A/B testing you never know the population mean, you’re estimating it, so always use the t-test. For N >100, the t-test numerically yields the same results as the z-test.

We investigate the difference between average time spent first. Again, the hypothesis:

H₀: “the average time spent is the same for the two versions”

H₁: “the average time spent is higher for version B”

In [7]:
#indicate a 1-tail test
# do a non-pool t-test by passing False into the equal_var to indicate the unequal var
statsmodels.stats.weightstats.ttest_ind(time_b,time_a,alternative='larger',usevar='unequal')

(6.887858875773088, 3.675691202742116e-11, 197.9495200941187)

We can see a p-value of close to 0(<0.05) which means there's a statistical difference and the average time spent on B is really longer than A.

## Z-test for evaluating difference in conversion 
For checking the difference in conversion rate, what test do we use? If we refer to the hypothesis testing flowchart again, we could see that we should use the 2 samples, z-test for proportions.

In [8]:
#https://www.statsmodels.org/stable/generated/statsmodels.stats.proportion.proportions_ztest.html
#https://sonalake.com/latest/hypothesis-testing-of-proportion-based-samples/
from statsmodels.stats.proportion import proportions_ztest
count = np.array([sum(pur_b), sum(pur_a)])
nobs = np.array([n_b, n_a])
proportions_ztest(count, nobs, alternative='larger')

(0.7071421391774789, 0.23973907562464403)

This time round, we could see a high p-value(>0.05). And this means that the observed differences is not statistically significant, could be due to random chance or simply not big enough.

## Repeating the test with a higher sample
If we increase the number of samples from 100 to 2000, guess what would happen?

In [12]:
#we generate a random sample under the binomial distribution which consists of 100 bernouili trials to simulate purchase(1/0) for each visit
n_a, n_b = 2500,2500
np.random.seed(888)
pur_a = np.random.binomial(n=1, p=0.5,size=n_a)
pur_b = np.random.binomial(n=1, p=0.55,size=n_b)
#generate random normally distribted samples for simulating time spent
time_a = np.random.normal(3, 1, n_a)
time_b = np.random.normal(4, 1, n_b)
grp_type_a = [0] * len(pur_a)
grp_type_b = [1] * len(pur_b)
dataset = pd.DataFrame({'treatment_grp':grp_type_a+grp_type_b,
                        'purchase':list(pur_a)+list(pur_b),
                       'time_spent':list(time_a)+list(time_b)}
                      )
ab_summary = dataset.pivot_table(values='purchase', index='treatment_grp', aggfunc=np.sum)
# add additional columns to the pivot table
ab_summary['total_visits'] = dataset.pivot_table(values='purchase', index='treatment_grp', aggfunc=lambda x: len(x))
ab_summary['avg_conv_rate'] = dataset.pivot_table(values='purchase', index='treatment_grp') #default=mean
ab_summary['avg_time_spent'] = dataset.pivot_table(values='time_spent', index='treatment_grp')
ab_summary

,purchase,total_visits,avg_conv_rate,avg_time_spent
treatment_grp,,,,
0,1277,2500,0.5108,3.008802
1,1342,2500,0.5368,4.007163


## T-test for evaluating average time spent

In [13]:
#indicate a 1-tail test
# do a non-pool t-test by passing False into the equal_var to indicate the unequal var
statsmodels.stats.weightstats.ttest_ind(time_b,time_a,alternative='larger',usevar='unequal')

(35.488940565432166, 1.556694190666638e-246, 4997.848116320836)

Obviously, with a higher sample, the difference between the average time spent doesn't change.

## Z-test for evaluating difference in conversion rate

In [14]:
count = np.array([sum(pur_b), sum(pur_a)])
nobs = np.array([n_b, n_a])
proportions_ztest(count, nobs, alternative='larger')

(1.8405639516201047, 0.03284274224174207)

With a higher sample, it has become signifcant, so we could see that the number of samples could actually change the outcome of the a/b testing and potentially translate to a different action that has business impact. So how do we know the right number of samples to experiment with?

# Choosing the number of samples for A/B testing


One major cost in every experiment is the time it takes to reach a statistically significant result. To estimate how long a given experiment will need to run to achieve statistical significance, we need to rely on power analysis to determine the optimal number of sample size. 

## Consideration for split proportion between control and treatmemt groups
We can naively choose a 50/50 split for control and treatment group, but practically we can't do that because we might risk the high converting traffic to the newer, lower converting pages. Depending on the risk appetitie, the company can split by 80/20 and even lower. However, if only 20% goes to treatment group, then we need a longer time to complete the test.

## Estimation of sample size via Power Analysis
A power analysis can be used to estimate the minimum sample size required for an experiment, given a desired significance level, effect size, and statistical power.

We need these information for calculation:

1) Baseline conversion rate (BCR)

2) Minimum detectable effect(MDE), which is essentially the sensitivity of your test. In other words, it is the smallest relative change in conversion rate you are interested in detecting. For example, if your baseline conversion rate is 20%, and you set an MDE of 10%, your test would detect any changes that move your conversion rate outside the absolute range of 18% to 22% (a 10% relative effect is a 2% absolute change in conversion rate in this example)

3) Power, is the probability of a hypothesis test of finding an effect if there is an effect to be found. The convention is to require 80% power. Power depends on magnitude of the change, and variance among samples.  It is essentially a measure of whether your test has adequate data to reach a conclusive result (True Positive), 1-Beta. By increasing sample size, we also increase the power, because the variance decreases.

<img src='https://miro.medium.com/max/1400/1*BPP6QYVIx4yqXrFfzUgTBg.png'/>
<img src='https://miro.medium.com/max/1400/1*8LVDtWvkXvuK2CXdF_zk-Q.png'/>

<img src='https://cdn.cms-twdigitalassets.com/content/dam/blog-twitter/archive/power_minimal_detectableeffectandbucketsizeestimationinabtests95.thumb.1280.1280.png'/>

In [34]:
import scipy.stats as scs
#http://ethen8181.github.io/machine-learning/ab_tests/frequentist_ab_test.html#Determining-Sample-Size
def min_sample_size(bcr, mde, power=0.8, sig_level=0.05):
    """Returns the minimum sample size to set up a split test

    Arguments:
        bcr (float): probability of success for control, sometimes
        referred to as baseline conversion rate

        mde (float): minimum change in measurement between control
        group and test group if alternative hypothesis is true, sometimes
        referred to as minimum detectable effect

        power (float): probability of rejecting the null hypothesis when the
        null hypothesis is false, typically 0.8

        sig_level (float): significance level often denoted as alpha,
        typically 0.05

    Returns:
        min_N: minimum sample size (float)

    References:
        Stanford lecture on sample sizes
        http://statweb.stanford.edu/~susan/courses/s141/hopower.pdf
    """
    # standard normal distribution to determine z-values
    standard_norm = scs.norm(0, 1)

    # find Z_beta from desired power
    Z_beta = standard_norm.ppf(power)

    # find Z_alpha
    Z_alpha = standard_norm.ppf(1-sig_level/2)

    # average of probabilities from both groups
    pooled_prob = (bcr + bcr+mde) / 2

    min_N = (2 * pooled_prob * (1 - pooled_prob) * (Z_beta + Z_alpha)**2
             / mde**2)

    return min_N

Note that the number of samples we need, is per group.

In [18]:
min_sample_size(bcr=0.10, mde=0.02, power=0.8, sig_level=0.05)

3842.026629963879

### Lowering MDE

In [19]:
min_sample_size(bcr=0.10, mde=0.0002, power=0.8, sig_level=0.05)

35351350.39906843

The lower the Minimum detectable effect(MDE), the more samples you need. It's easier to observe large difference with a small sample due to chance, but to detect a small effect, you would need a larger sample size.

### Increasing power

In [23]:
min_sample_size(bcr=0.10, mde=0.02, power=0.9, sig_level=0.05)

5143.383588575183

The higher the Statistical power, the more samples you need. Reason: https://web.ma.utexas.edu/users/mks/statmistakes/powersamplesize.html

### Decreasing significant level

In [21]:
min_sample_size(bcr=0.10, mde=0.02, power=0.8, sig_level=0.02)

4912.635836596197

Also, the lower the significance level(alpha), the more samples you need. If the effect size is small, we need a large sample size in order to detect the difference otherwise the effect will be masked by the randomness in the samples.

<img src="https://i1.wp.com/www.theanalysisfactor.com/wp-content/uploads/2009/12/one-sided-test.jpg?w=579&ssl=1" width=400/>
https://www.theanalysisfactor.com/confusing-statistical-terms-1-alpha-and-beta/

Back to case, how much we need?

We know that currently conv rate is 50%, we only want to consider an improvement if CR increase by another 5%(to 55%).

Also, we want a power of 80%, which means probabilty of rejecting null hypothesis when it's false, or the true positive rate.

In [63]:
min_sample_size(bcr=0.50, mde=0.05, power=0.8, sig_level=0.1)

1233.420167787943

In [64]:
#we generate a random sample under the binomial distribution which consists of 100 bernouili trials to simulate purchase(1/0) for each visit
n_a, n_b = 1233,1233
np.random.seed(888)
pur_a = np.random.binomial(n=1, p=0.5,size=n_a)
pur_b = np.random.binomial(n=1, p=0.55,size=n_b)
#generate random normally distribted samples for simulating time spent
time_a = np.random.normal(3, 1, n_a)
time_b = np.random.normal(4, 1, n_b)
grp_type_a = [0] * len(pur_a)
grp_type_b = [1] * len(pur_b)
dataset = pd.DataFrame({'treatment_grp':grp_type_a+grp_type_b,
                        'purchase':list(pur_a)+list(pur_b),
                       'time_spent':list(time_a)+list(time_b)}
                      )
ab_summary = dataset.pivot_table(values='purchase', index='treatment_grp', aggfunc=np.sum)
# add additional columns to the pivot table
ab_summary['total'] = dataset.pivot_table(values='purchase', index='treatment_grp', aggfunc=lambda x: len(x))
ab_summary['rate'] = dataset.pivot_table(values='purchase', index='treatment_grp') #default=mean
ab_summary['avg_time_spent'] = dataset.pivot_table(values='time_spent', index='treatment_grp')
ab_summary

,purchase,total,rate,avg_time_spent
treatment_grp,,,,
0,642,1233,0.520681,3.022151
1,682,1233,0.553122,3.999009


In [65]:
#https://www.statsmodels.org/stable/generated/statsmodels.stats.proportion.proportions_ztest.html
#https://sonalake.com/latest/hypothesis-testing-of-proportion-based-samples/
import numpy as np
from statsmodels.stats.proportion import proportions_ztest
count = np.array([sum(pur_b), sum(pur_a)])
nobs = np.array([n_b, n_a])
proportions_ztest(count, nobs, alternative='larger')

(1.6153977846315681, 0.053112289847787646)

## Interaction between sample size and power
<img src='ab_testing/decrease_sample_size.png'>
Decrease sample size = variance increase and widens.
<img src='ab_testing/increase_sample_size.png'>
Increase sample size = variance decrease and narrows.